In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/kaggle/input/unemployment-in-india/Unemployment_Rate_upto_11_2020.csv')

df.head()

In [ ]:
df.columns = df.columns.str.strip()

In [ ]:
df.drop(['longitude', 'latitude'], axis='columns', inplace=True)

df.head()

In [ ]:
df = df.rename(columns={'Region':'States', 'Region.1':'Region'})

df.head()

In [ ]:
df.drop('Frequency', axis='columns', inplace=True)

df.head()

In [ ]:
df.describe()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

df.info()

In [ ]:
df['Month'] = df['Date'].dt.month

df.drop('Date', axis='columns', inplace=True)

df.head()

In [ ]:
df['States'].value_counts()

In [ ]:
df['Region'].value_counts()

In [ ]:
plt.figure(figsize=(17, 11))
sns.countplot(data=df, x='Region')

In [ ]:
plt.figure(figsize=(16, 10))
sns.boxplot(data=df, y='States', x='Estimated Unemployment Rate (%)')

In [ ]:
aur_df = df[['States', 'Estimated Unemployment Rate (%)']].groupby('States').mean().reset_index()
aur_df = aur_df.sort_values(by='Estimated Unemployment Rate (%)', ascending=False)

aur_df.head()

In [ ]:
plt.figure(figsize=(16, 10))
sns.barplot(data=aur_df, y='Estimated Unemployment Rate (%)', x='States')

In [ ]:
aur_df = df[['Region', 'Estimated Unemployment Rate (%)']].groupby('Region').mean().reset_index()
aur_df = aur_df.sort_values(by='Estimated Unemployment Rate (%)')

aur_df.head()

In [ ]:
plt.figure(figsize=(16, 10))
sns.barplot(data=aur_df, x='Estimated Unemployment Rate (%)', y='Region', errorbar=None)
region_colors = {0: 'yellow', 1: 'red', 2: 'green', 3:'blue', 4:'maroon'}


In [ ]:
bf_lock_df = df[(df['Month'] >= 1) & (df['Month'] <= 3)][['States', 'Estimated Unemployment Rate (%)']]
af_lock_df = df[(df['Month'] >= 3) & (df['Month'] <= 5)][['States', 'Estimated Unemployment Rate (%)']]

In [ ]:
bf_lock_df = bf_lock_df.groupby('States')['Estimated Unemployment Rate (%)'].mean().reset_index().rename(columns={'Estimated Unemployment Rate (%)':'Unemployment Rate before Lock-Down'})
af_lock_df = af_lock_df.groupby('States')['Estimated Unemployment Rate (%)'].mean().reset_index().rename(columns={'Estimated Unemployment Rate (%)':'Unemployment Rate after Lock-Down'})

In [ ]:
bf_lock_df['Percentage Change in Unemployment'] = round((af_lock_df['Unemployment Rate after Lock-Down'] - bf_lock_df['Unemployment Rate before Lock-Down']) / bf_lock_df['Unemployment Rate before Lock-Down'], 2)

In [ ]:
bf_lock_df['Unemployment Rate after Lock-Down'] = af_lock_df['Unemployment Rate after Lock-Down']

In [ ]:
plot_df = bf_lock_df.sort_values('Percentage Change in Unemployment', ascending=False)

In [ ]:
plt.figure(figsize=(30, 15))
sns.barplot(data=plot_df, y='Percentage Change in Unemployment', x='States')

In [ ]:
def sort_impact_level(x):
    if x > 20:
        return 'hardest impacted states'
    elif x > 10:
        return 'harder impacted states'
    elif x > 5:
        return 'hard impacted states'
    elif x > 0:
        return 'impacted states'
    else:
        return 'not impacted states'

In [ ]:
plot_df['Impact Status'] = plot_df['Percentage Change in Unemployment'].apply(lambda x:sort_impact_level(x))

In [ ]:
plt.figure(figsize=(16, 10))
sns.barplot(data=plot_df, y='Percentage Change in Unemployment', x='Percentage Change in Unemployment', hue='States', width=1.9)